<a href="https://colab.research.google.com/github/BabeRush/NLP_final_project/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

## Notebook preparation

### Configurations

In [1]:
# Checking high RAM from google
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# Assigning and checking for GPU usage
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [6]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

### Imports & Installations

In [4]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00


In [5]:
# IMPORTS
# General
import warnings
import random

# Math & Data organization
import numpy as np
import scipy as sp
import pandas as pd
import re

# Deep Learning
import transformers
from transformers import AutoModel, BertTokenizerFast , BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Visualization
import seaborn as sns
from tqdm import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt

# Working with files
from google.colab import files
import pickle

## Creating the datasets

In [7]:
# Clone the datasets
# Description: ************TODO-LINK******************
!git clone https://github.com/alonbarnathan3/RUNI-NLP-FINAL-PROJECT.git

Cloning into 'RUNI-NLP-FINAL-PROJECT'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 21 (delta 0), reused 1 (delta 0), pack-reused 19
Receiving objects: 100% (21/21), 61.70 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (11/11), done.


In [8]:
df_real = pd.read_csv('/content/RUNI-NLP-FINAL-PROJECT/True.csv',encoding='utf-8')
df_fake = pd.read_csv ('/content/RUNI-NLP-FINAL-PROJECT/Fake.csv',encoding='utf-8')

# reading these tsv files: seperated by tabs, no header
liar_train = pd.read_csv('/content/RUNI-NLP-FINAL-PROJECT/train.tsv', sep='\t',encoding='utf-8', header=None)
liar_test = pd.read_csv('/content/RUNI-NLP-FINAL-PROJECT/test.tsv', sep='\t',encoding='utf-8', header=None)
liar_valid = pd.read_csv('/content/RUNI-NLP-FINAL-PROJECT/valid.tsv', sep='\t',encoding='utf-8', header=None)

# Rename the columns to match the 'df_real' dataset
liar_train.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']
liar_valid.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']
liar_test.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']

### Filter missing values (statistics below)

In [9]:
temp_real = len(df_real)
temp_fake = len(df_fake)
temp_li_train = len(liar_train)
temp_li_val = len(liar_valid)
temp_li_test = len(liar_test)

df_real = df_real.dropna()
df_fake = df_fake.dropna()
liar_train = liar_train.dropna()
liar_valid = liar_valid.dropna()
liar_test = liar_test.dropna()

g1 = temp_real - len(df_real)
g2 = temp_fake - len(df_fake)
g3 = temp_li_train - len(liar_train)
g4 = temp_li_val - len(liar_valid)
g5 = temp_li_test - len(liar_test)

print(f'Number of df_real sentences: {len(df_real)}, removed {g1}({(g1/temp_real)*100:.1f}%) instances')
print(f'Number of df_fake sentences: {len(df_fake)}, removed {g2}({(g1/temp_fake)*100:.1f}%) instances')
print(f'Number of liar_train sentences: {len(liar_train)}, removed {g3}({(g3/temp_li_train)*100:.1f}%) instances')
print(f'Number of liar_valid sentences: {len(liar_valid)}, removed {g4}({(g4/temp_li_val)*100:.1f}%) instances')
print(f'Number of liar_test sentences: {len(liar_test)} removed {g5}({(g5/temp_li_test)*100:.1f}%) instances')

Number of df_real sentences: 21417, removed 0(0.0%) instances
Number of df_fake sentences: 23481, removed 0(0.0%) instances
Number of liar_train sentences: 6724, removed 3516(34.3%) instances
Number of liar_valid sentences: 861, removed 423(32.9%) instances
Number of liar_test sentences: 853 removed 414(32.7%) instances


## Datasets combinations